In [1]:
import pandas as pd
from druxai.utils.data import DrugResponseDataset

In [2]:
df = pd.read_csv("/Users/niklaskiermeyer/Desktop/Codespace/DruxAI/data/raw/sanger_files/sanger-dose-response.csv")
data_prism = DrugResponseDataset("/Users/niklaskiermeyer/Desktop/Codespace/DruxAI/data/preprocessed")

INFO     Loaded targets with shape: (556840, 8)                                                                    
INFO     Loaded molecular data with shape: (1479, 19193)                                                           


In [3]:
df = df[~df["auc"].isna()].reset_index(drop=True)
df

,DATASET,COSMIC_ID,DRUG_ID,MIN_CONC,MAX_CONC,RMSE_PUBLISHED,Z_SCORE_PUBLISHED,IC50_PUBLISHED,AUC_PUBLISHED,ARXSPAN_ID,DRUG_NAME,BROAD_ID,upper_limit,ec50,slope,lower_limit,auc,log2.ic50,mse,R2
0,GDSC1,683665,1,0.007813,2.0,0.022518,-0.192056,10.977393,0.982116,ACH-002270,ERLOTINIB,BRD-K70401845,0.992788,2.839376e+00,-5.670993,0.514389,0.990834,NaN,0.000034,0.904675
1,GDSC1,684055,1,0.007813,2.0,0.031831,0.505823,23.133991,0.984820,ACH-002104,ERLOTINIB,BRD-K70401845,1.006405,2.864875e-02,-0.186377,0.990054,0.997138,NaN,0.000057,0.028903
2,GDSC1,684062,1,0.007813,2.0,0.087010,-0.114395,11.926884,0.944463,ACH-002111,ERLOTINIB,BRD-K70401845,0.989580,7.580375e-02,-12.222777,0.894027,0.933185,NaN,0.000623,0.777093
3,GDSC1,684072,1,0.007813,2.0,0.016288,-0.530674,7.645605,0.950763,ACH-000087,ERLOTINIB,BRD-K70401845,0.998887,9.671393e-01,-2.496776,0.816721,0.973032,NaN,0.000018,0.993758
4,GDSC1,687799,1,0.007813,2.0,0.043985,-5.365666,0.043694,0.349947,ACH-000766,ERLOTINIB,BRD-K70401845,0.939255,9.351034e-02,-0.698966,0.047188,0.458337,-3.481481,0.000693,0.987013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327335,GDSC2,905951,2172,0.009766,10.0,0.143863,0.042524,25.410793,0.877741,ACH-000288,JQ1,"BRD-K54606188, NA",3.929045,3.287745e+11,-0.012190,-3.449953,0.856099,NaN,0.006178,0.292447
327336,GDSC2,906862,2172,0.009766,10.0,0.088631,-2.223819,0.339325,0.510590,ACH-001065,JQ1,"BRD-K54606188, NA",0.935866,4.096968e-01,-2.014115,0.176515,0.585800,-1.073816,0.003198,0.971991
327337,GDSC2,907046,2172,0.009766,10.0,0.114748,-0.578942,7.780877,0.843211,ACH-000930,JQ1,"BRD-K54606188, NA",0.960799,3.695946e-01,-1.161533,0.687792,0.830671,NaN,0.002792,0.795935
327338,GDSC2,749709,2172,0.009766,10.0,0.047625,1.642265,534.688321,0.983634,ACH-000859,JQ1,"BRD-K54606188, NA",0.977260,1.161541e+30,-0.031889,0.900234,0.969870,NaN,0.000879,0.000540


In [4]:
selected_columns = df[["DATASET", "DRUG_ID", "IC50_PUBLISHED", "AUC_PUBLISHED", "ARXSPAN_ID", "DRUG_NAME"]]

In [5]:
# Find NAs of specific columns
selected_columns.isna().sum()

DATASET              0
DRUG_ID              0
IC50_PUBLISHED       0
AUC_PUBLISHED        0
ARXSPAN_ID        7227
DRUG_NAME            0
dtype: int64

In [6]:
# Drop rows with NAs
sanger_dataset = selected_columns.dropna().copy()

# Create an auc per drug column
sanger_dataset["auc_per_drug"] = sanger_dataset.groupby("DRUG_ID")["AUC_PUBLISHED"].transform(
    lambda x: (x - x.mean()) / x.std())

sanger_dataset = sanger_dataset.rename(columns={"DATASET": "Dataset", "DRUG_ID": "Drug_ID", "IC50_PUBLISHED": "IC50",
                                                "AUC_PUBLISHED": "auc", "ARXSPAN_ID": "cell_line", "DRUG_NAME": "DRUG"})

sanger_dataset = sanger_dataset.reindex(columns=["cell_line", "DRUG", "auc", "IC50", "auc_per_drug", "Drug_ID",
                                                 "Dataset"])

In [7]:
count_cell_lines_matches = len(set(sanger_dataset["cell_line"]) & set(data_prism.targets["cell_line"]))
print(f"Matching cell lines in prism and sanger data: {count_cell_lines_matches}")
print(f"There are a total of {sanger_dataset['cell_line'].nunique()} cell lines in the sanger dataset.")

Matching cell lines in prism and sanger data: 343
There are a total of 972 cell lines in the sanger dataset.


In [8]:
# Remove cell lines which are already in the prism dataset
sanger_dataset = sanger_dataset[~sanger_dataset["cell_line"].isin(data_prism.targets["cell_line"])]

In [9]:
# Check whether all cell lines have a match in the gene expression data
matches = len(set(data_prism.molecular_data.index.values) & set(sanger_dataset["cell_line"]))
print(f"There are a total of {matches} matches with gene expression data.")

# Only keep cell lines where we have a match
sanger_dataset = sanger_dataset[~sanger_dataset["cell_line"].isin(list(set(data_prism.molecular_data.index.values)))]

There are a total of 354 matches with gene expression data.


In [25]:
# There are multiple Drug_IDs for the same drug!!
grouped_by_drug = sanger_dataset.groupby("DRUG")[["cell_line", "DRUG", "Drug_ID"]]

# Iterate over each group
for counter, (drug, group_df) in enumerate(grouped_by_drug):
    if counter == 20:
        break
    # Check if all DRUG_ID values within the group are the same
    if group_df["Drug_ID"].nunique() == 1:
        continue
    else:
        print(f"Drug_ID values for {drug} are not consistent: {group_df['Drug_ID'].unique()}")

Drug_ID values for 5-FLUOROURACIL are not consistent: [ 179 1073]
Drug_ID values for AFATINIB are not consistent: [1032 1377]
Drug_ID values for AGI-6780 are not consistent: [ 562 1634]
Drug_ID values for AKT INHIBITOR VIII, KIN001-102 are not consistent: [171 228]
Drug_ID values for ALISERTIB are not consistent: [ 431 1051]


In [32]:
# Check if all drugs from saner dataset are in the prism dataset and vice versa.
drugs_in_sanger = set(sanger_dataset["DRUG"])
drugs_in_prism = set(data_prism.targets["DRUG"])

print(f"Drugs in sanger dataset but not in prism dataset: {drugs_in_sanger - drugs_in_prism}")
print(f"Drugs in prism dataset but not in sanger dataset: {drugs_in_prism - drugs_in_sanger}")

print(f"There are a total of {len(drugs_in_sanger - drugs_in_prism)} unique drugs in sanger which are not in prism.")
print(f"There are a total of {len(drugs_in_prism - drugs_in_sanger)} unique drugs in prism which are not in sanger.")

Drugs in sanger dataset but not in prism dataset: {'MIRIN', 'AZD3759', 'IC-87114', 'RU-SKI-43', 'TL-1-85', 'CD532', 'BRIVANIB', 'BMS-509744, KIN001-127', 'T0901317', 'LJI308', 'PIK-93', 'RTRAIL', 'KIN001-206', 'MIM1, MIM 1, MIM-1', 'AGI-5198', 'BPD-00008900', 'DIHYDROROTENONE', 'NSC-87877', 'XAV-939', 'KRAS (G12C) INHIBITOR-12, GTPL8020', 'CAY10566', 'QL-VIII-58', 'CAY10603', 'MITOMYCIN-C', 'AZD6738', 'VENOTOCLAX, ABT-199, VENECLEXTA, GDC-0199', 'SCH772984, CS-1421', 'PF-4708671', 'KIN001-260, BAYER IKKB INHIBITOR', 'GSK2606414', 'MN-64', 'HG-5-88-01', 'DORAMAPIMOD', 'JNJ-38877605', 'PEMETREXED', 'CPI-613', 'LGK-974', 'MK-8776, SCH900776', 'GSK1059615', 'WZ4003', 'XMD11-85H', 'AS601245', 'WHI-P97, KIN001-055', 'DACTINOMYCIN', 'ZIBOTENTAN', 'I-BRD9', 'S-TRITYL-L-CYSTEINE', 'NUTLIN-3A (-)', 'JW-7-52-1, NA', 'Z-LLNLE-CHO, Z-L-NORLEUCINE-CHO', 'AZD5582', 'AKT INHIBITOR VIII, KIN001-102', 'QL-XII-61', 'RUCAPARIB', 'GNE-317, GNE317, GNE 317', 'CCT007093', 'QS11', 'GW843682X', 'ML323, GTPL789

In [36]:
# Save newly created dataset
sanger_dataset.to_csv("/Users/niklaskiermeyer/Desktop/Codespace/DruxAI/data/preprocessed/sanger_target.csv",
                      index=False)

In [13]:
# Find matches between gene Databank and cell line
data_prism.targets

,cell_line,DRUG,auc,logauc,auc_per_drug,means,stds,range
0,ACH-000879,CYTARABINE,1.677789,0.517477,2.367402,0.228684,0.121987,0.782691
1,ACH-000320,CYTARABINE,1.240300,0.215353,-0.109281,0.228684,0.121987,0.782691
2,ACH-001145,CYTARABINE,1.472333,0.386848,1.296560,0.228684,0.121987,0.782691
3,ACH-000873,CYTARABINE,1.207160,0.188270,-0.331294,0.228684,0.121987,0.782691
4,ACH-000855,CYTARABINE,1.229332,0.206471,-0.182097,0.228684,0.121987,0.782691
...,...,...,...,...,...,...,...,...
556835,ACH-001321,BREQUINAR,0.987409,-0.012671,0.979322,-0.184222,0.175173,1.078361
556836,ACH-001321,AZD8931,1.000000,0.000000,0.879937,-0.199376,0.226580,2.297688
556837,ACH-001321,AZD2014,1.000000,0.000000,1.643680,-0.279217,0.169873,1.025072
556838,ACH-001321,MOTESANIB,0.888088,-0.118685,-0.367226,-0.084827,0.092199,0.957449
